#### 获得初始电影实体

In [1]:
# 读取 id 列表
import csv
import pickle

origin_movies = set()
def entry0():

    # 构建实体-tag字典，同时匹配获得Freebase中对应的实体（共578个可匹配实体），加入到一跳可匹配实体
    with open('lab2/Stage1/douban2fb.txt', 'rb') as f:
        for line in f:
            line = line.strip()
            parts = line.decode().split('\t')
            origin_movies.add(parts[1])
    print("entry0:", len(origin_movies))
    with open("lab2/Stage1/entry0.pkl", "wb") as f:
        pickle.dump(origin_movies, f)


entry0()


entry0: 578


#### 一跳得到子图并对实体和关系计数

In [2]:
import gzip
import pickle
from tqdm import tqdm

template_str = "http://rdf.freebase.com/ns/"
freebase_info_fpath = "lab2/Stage1/freebase_douban.gz"  # 初始freebase
outfile1 = "lab2/Stage1/graph_1step.gz"  # 一跳输出文件
outfile2 = "lab2/Stage1/graph_2step.gz"  # 二跳输出文件
outfile3 = "lab2/Stage1/graph_3step.gz"

entry_num = {}
def step1():
    # 以 mvi_entities 为起点生成一跳子图，保存到 grarph1step.gz 文件中
    with gzip.open(outfile1, 'wb') as ans:
        with open('lab2/Stage1/entry0.pkl', 'rb') as f1:
            mvi_entities = pickle.load(f1)
        with gzip.open(freebase_info_fpath, 'rb') as f:
            
            for line in tqdm(f, total=395577070):
                # if count > max_run_times:
                #     break  # 超过设定上限即停止查找一跳可达实体
                line = line.strip()
                triple_parts = line.decode().split('\t')
                if (template_str not in triple_parts[0]
                        or template_str not in triple_parts[1]
                        or template_str not in triple_parts[2]):
                    continue
                # 头实体
                head = triple_parts[0][len(template_str) + 1:].strip('>')
                # 关系
                rel = triple_parts[1][len(template_str) + 1:].strip('>')
                # 尾实体
                tail = triple_parts[2][len(template_str) + 1:].strip('>')

                # 保存头实体在 mvi_entities 中的三元组
                if head in mvi_entities:
                    ans.write(line + b'\n')
                    # 实体相关三元组计数
                    if (head in entry_num):
                        entry_num[head] += 1
                    else:
                        entry_num[head] = 1
                    if (tail in entry_num):
                        entry_num[tail] += 1
                    else:
                        entry_num[tail] = 1
                    if (rel in entry_num):
                        entry_num[rel] += 1
                    else:
                        entry_num[rel] = 1


step1()

100%|██████████| 395577070/395577070 [10:28<00:00, 629609.93it/s]


#### 筛选一跳子图

In [18]:
def Select1():
    with gzip.open(outfile1, 'rb') as f:
        mvi_entities2 = set()
        with open("lab2/Stage1/entry0.pkl", "rb") as f1:
            mvi_entities = pickle.load(f1)
        count = 0
        for line in tqdm(f, total=395577070):
            triple_parts = line.decode().split('\t')[:3]
            # 头实体
            head = triple_parts[0][len(template_str) + 1:].strip('>')
            # 关系
            rel = triple_parts[1][len(template_str) + 1:].strip('>')
            # 尾实体
            tail = triple_parts[2][len(template_str) + 1:].strip('>')
            if ((head in mvi_entities or entry_num[head] > 20)
                    and (entry_num[tail] > 20)
                    and (rel in mvi_entities or entry_num[rel] > 50)):
                mvi_entities2.add(head)
                mvi_entities2.add(tail)
            count += 1 
        print("1step:", count)
        print("entry1:", len(mvi_entities2))
        with open("lab2/Stage1/entry1.pkl", "wb") as f:
            pickle.dump(mvi_entities2, f)
        # print(mvi_entities2)


Select1()

  0%|          | 118773/395577070 [00:00<13:29, 488598.43it/s]

1step: 118773
entry1: 758


#### 二跳子图

In [19]:
entry_num2 = {}  # 二跳元素存储


def step2():
    with gzip.open(outfile2, 'wb') as ans:
        with open('lab2/Stage1/entry1.pkl', 'rb') as f1:
            mvi_entities = pickle.load(f1)
        with gzip.open(freebase_info_fpath, 'rb') as f:
            for line in tqdm(f, total=395577070):
                # if count > max_run_times:
                #     break  # 超过设定上限即停止查找一跳可达实体
                line = line.strip()
                triple_parts = line.decode().split('\t')
                if (template_str not in triple_parts[0]
                        or template_str not in triple_parts[1]
                        or template_str not in triple_parts[2]):
                    continue
                # 头实体
                head = triple_parts[0][len(template_str) + 1:].strip('>')
                # 关系
                rel = triple_parts[1][len(template_str) + 1:].strip('>')
                # 尾实体
                tail = triple_parts[2][len(template_str) + 1:].strip('>')

                # 保存头实体在 mvi_entities 中的三元组
                if head in mvi_entities:
                    ans.write(line + b'\n')
                    # 实体相关三元组计数
                    if (head in entry_num2):
                        entry_num2[head] += 1
                    else:
                        entry_num2[head] = 1
                    if (tail in entry_num2):
                        entry_num2[tail] += 1
                    else:
                        entry_num2[tail] = 1
                    if (rel in entry_num2):
                        entry_num2[rel] += 1
                    else:
                        entry_num2[rel] = 1


step2()

100%|██████████| 395577070/395577070 [18:26<00:00, 357411.87it/s] 


#### 第一次筛选二跳子图

In [20]:
def Select2():
    count = 0
    with gzip.open(outfile2, 'rb') as f:
        mvi_entities2 = set()
        with open("lab2/Stage1/entry1.pkl", "rb") as f1:
            mvi_entities = pickle.load(f1)
        for line in tqdm(f, total=104715852):
            triple_parts = line.decode().split('\t')[:3]
            # 头实体
            head = triple_parts[0][len(template_str) + 1:].strip('>')
            # 关系
            rel = triple_parts[1][len(template_str) + 1:].strip('>')
            # 尾实体
            tail = triple_parts[2][len(template_str) + 1:].strip('>')
            if ((head in mvi_entities or entry_num2[head] <= 20000)
                    and (entry_num2[tail] <= 20000)
                    and (rel in mvi_entities or entry_num2[rel] > 50)):
                mvi_entities2.add(head)
                mvi_entities2.add(tail)
                count += 1
        print('step2:',count)
        print("entry2:", len(mvi_entities2))
        with open("lab2/Stage1/entry2.pkl", "wb") as f:
            pickle.dump(mvi_entities2, f)


Select2()

100%|██████████| 104715852/104715852 [05:06<00:00, 341413.53it/s]


step2: 104704830
entry2: 50231562


#### 对筛选过的二跳子图重新计数

In [21]:
entry_num3 = {}  # 二跳首次筛选后元素存储
def step3():
    with gzip.open(outfile3, 'wb') as ans:
        with open('lab2/Stage1/entry2.pkl', 'rb') as f1:
            mvi_entities = pickle.load(f1)
        with gzip.open(outfile2, 'rb') as f:
            for line in tqdm(f, total=104715852):
                # if count > max_run_times:
                #     break  # 超过设定上限即停止查找一跳可达实体
                line = line.strip()
                triple_parts = line.decode().split('\t')
                if (template_str not in triple_parts[0]
                        or template_str not in triple_parts[1]
                        or template_str not in triple_parts[2]):
                    continue
                # 头实体
                head = triple_parts[0][len(template_str) + 1:].strip('>')
                # 关系
                rel = triple_parts[1][len(template_str) + 1:].strip('>')
                # 尾实体
                tail = triple_parts[2][len(template_str) + 1:].strip('>')

                # 保存头实体在 mvi_entities 中的三元组
                if head in mvi_entities:
                    ans.write(line + b'\n')
                    # 实体相关三元组计数
                    # if (head in entry_num3):
                    #     entry_num3[head] += 1
                    # else:
                    #     entry_num3[head] = 1
                    # if (tail in entry_num3):
                    #     entry_num3[tail] += 1
                    # else:
                    #     entry_num3[tail] = 1
                    # if (rel in entry_num3):
                    #     entry_num3[rel] += 1
                    # else:
                    #     entry_num3[rel] = 1


step3()

100%|██████████| 104715852/104715852 [09:07<00:00, 191128.85it/s]


#### 迭代筛选

In [2]:
def Select(triplets):
    entry_num = {}
    for triplet in triplets:
        if (triplet[0] not in entry_num):
            entry_num[triplet[0]] = 0
        entry_num[triplet[0]] += 1
        if (triplet[2] not in entry_num):
            entry_num[triplet[2]] = 0
        entry_num[triplet[2]] += 1
        if (triplet[1] not in entry_num):
            entry_num[triplet[1]] = 0
        entry_num[triplet[1]] += 1
    ans = []
    for triplet in triplets:
        if ((triplets[0] in origin_movies or entry_num[triplet[0]]>15)and(triplets[2] in origin_movies or entry_num[triplet[2]]>15)and(entry_num[triplet[1]]>50)):
            ans.append(triplet)
    return ans

#### 二次筛选二跳子图

In [3]:
import gzip
import pickle
from tqdm import tqdm

template_str = "http://rdf.freebase.com/ns/"
freebase_info_fpath = "lab2/Stage1/freebase_douban.gz"  # 初始freebase
outfile1 = "lab2/Stage1/graph_1step.gz"  # 一跳输出文件
outfile2 = "lab2/Stage1/graph_2step.gz"  # 二跳输出文件
outfile3 = "lab2/Stage1/graph_3step.gz"


outfile3 = "lab2/Stage1/graph_3step.gz"


def Select3():
    before_selected = []
    with gzip.open(outfile3, 'rb') as f:
        for line in tqdm(f, total=104715852):
            triple_parts = line.decode().split('\t')[:3]
            # 头实体
            head = triple_parts[0][len(template_str) + 1:].strip('>')
            # 关系
            rel = triple_parts[1][len(template_str) + 1:].strip('>')
            # 尾实体
            tail = triple_parts[2][len(template_str) + 1:].strip('>')
            before_selected.append((head, rel, tail))
        print('get before_selected')
        selected = Select(before_selected)
        print('筛选后:', len(selected), '筛选前:', len(before_selected))

    while len(selected) < len(before_selected):  # 迭代筛选
        before_selected = selected
        selected = Select(before_selected)
        print('筛选后:', len(selected), '筛选前:', len(before_selected))

    with gzip.open('lab2/Stage1/final2.gz', 'wb') as ans:
        count = 0
        mvi_entities2 = set()
        for triplet in selected:
            mvi_entities2.add(triplet[0])
            mvi_entities2.add(triplet[2])
        print("final:", len(selected))
        print("entry3:", len(mvi_entities2))
        with open("lab2/Stage1/entry3.pkl", "wb") as f:
            pickle.dump(mvi_entities2, f)


Select3()

100%|██████████| 104715852/104715852 [04:57<00:00, 352524.46it/s]


get before_selected
筛选后: 359968 筛选前: 104715852
筛选后: 359481 筛选前: 359968
筛选后: 359279 筛选前: 359481
筛选后: 359187 筛选前: 359279
筛选后: 359132 筛选前: 359187
筛选后: 359101 筛选前: 359132
筛选后: 359080 筛选前: 359101
筛选后: 359047 筛选前: 359080
筛选后: 359047 筛选前: 359047
final: 359047
entry3: 17674
